In [2]:
import pandas as pd
import os
import json
from datetime import datetime,timedelta
from tqdm import tqdm
import numpy as np
import itertools

In [3]:
folder = 'data'

# Historical data exploration

In [4]:
bikes_data = pd.read_csv(os.path.join(folder,'bike_list.csv'))
places = pd.read_csv(os.path.join(folder,'places.csv'))
cities = pd.read_csv(os.path.join(folder,'cities.csv'))
countries = pd.read_csv(os.path.join(folder,'countries.csv'))

bikes_data['appeared'] = pd.to_datetime(bikes_data['appeared'], format='%Y-%m-%d %H:%M:%S')
bikes_data['disappeared'] = pd.to_datetime(bikes_data['disappeared'], format='%Y-%m-%d %H:%M:%S')

## Functions

In [26]:
def get_data_from_city(city_id):
    '''get bikes data from the specific city 
        result data are merged bikes data and places -> to get lat, lng and uid of a station
    '''
    city_stations_guids = list(places[places['cityguid']==city_id]['guid'])
    city_bikes = bikes_data[(bikes_data['placeguid'].isin(city_stations_guids))]
    city_bikes = city_bikes.merge(places[['guid','uid','lat','lng']], left_on='placeguid', right_on='guid', suffixes=['_bikes','_stations'])
    return city_bikes

In [34]:
def datetime_range(start, end, delta):
    current = start
    while current < end:
        yield current
        current += delta


In [43]:
def get_time_index(start,stop,minutes_in_interval):
    _ = list(datetime_range(start, stop, timedelta(minutes=minutes_in_interval)))
    return _

In [46]:
def prepere_data(data,time_index):

    # initial filtering
    data = data[(data['appeared']>min(time_index)) & (data['appeared']<max(time_index))]
    data_uid = list(np.unique(data['uid']))

    # creating index
    tuples = list(itertools.product(time_index,data_uid))
    index = pd.MultiIndex.from_tuples(tuples, names=["time","station"])

    # producting data
    results = pd.DataFrame(data = np.zeros(len(index)),index=index, columns = ['number_of_bikes'])
    for i in tqdm(index):
        t,station = i
        station_bikes = data[data['uid']==station]
        results.loc[i]=len(station_bikes[(station_bikes['appeared']<t) & (station_bikes['disappeared']>t)])
    return results

## Countries
- guid - id of the country in other tables
- country - code
- country_name - country name
- lat,lng - coordinates
- created - probably when the first recird from this country was added to the db

In [4]:
countries.head()

,guid,country,country_name,lat,lng,created
0,1,DE,Germany,50.7086,10.63480,2019-05-23 20:03:43.310
1,2,AT,Austria,47.3835,13.13960,2019-05-23 20:04:07.830
2,3,NZ,New Zealand,-39.0789,175.64900,2019-05-23 20:04:07.937
3,4,CH,Switzerland,47.2569,8.62976,2019-05-23 20:04:26.660
4,5,LV,Latvia,56.9704,23.97490,2019-05-23 20:04:52.500


## Cities
* guid - unique id used in ither tables
* uid -  id from API-> sometimes the same for different cities (examples shown later)
* name - name of the city -> same issue
* countryguid - unique id of the country
* lat, lng - coordinates
* created - probably when the first recird from this city was added to the db

In [5]:
cities.head()

,guid,uid,name,countryguid,lat,lng,created
0,1,7,Wiesbaden,1,50.0709,8.24322,2019-05-23 20:03:43.343
1,2,43,Hamburg,1,53.5506,9.99052,2019-05-23 20:03:46.920
2,3,101,Tübingen,1,48.5203,9.05591,2019-05-23 20:03:51.463
3,4,139,München,1,48.1358,11.53560,2019-05-23 20:03:52.790
4,5,147,Flensburg,1,54.7804,9.43571,2019-05-23 20:03:53.080


In [9]:
cities.describe()

,guid,uid,countryguid,lat,lng
count,281.000000,281.000000,281.000000,281.00000,281.000000
mean,141.000000,392.960854,6.850534,47.29513,13.673814
std,81.261922,174.429873,6.542095,11.08468,27.661095
min,1.000000,1.000000,1.000000,-43.53410,-123.361000
25%,71.000000,254.000000,1.000000,46.99050,8.340980
50%,141.000000,424.000000,6.000000,49.80000,15.235600
75%,211.000000,537.000000,10.000000,51.61060,18.946300
max,281.000000,648.000000,25.000000,65.01650,174.763000


Not unique uid

In [6]:
cities[cities['uid'] == 548]

,guid,uid,name,countryguid,lat,lng,created
199,200,548,Tarnów,6,50.0113,20.9728,2019-05-23 20:23:14.697
280,281,548,Warszawa,6,50.0113,20.9728,2019-10-09 10:04:29.167


In [7]:
cities[cities['uid']==362]

,guid,uid,name,countryguid,lat,lng,created
143,144,362,Berlin,1,52.5087,13.3563,2019-05-23 20:14:25.917


Not unique name

In [8]:
cities[cities['name']=='Warszawa']

,guid,uid,name,countryguid,lat,lng,created
66,67,210,Warszawa,6,52.2265,21.0127,2019-05-23 20:06:29.480
280,281,548,Warszawa,6,50.0113,20.9728,2019-10-09 10:04:29.167


## Places
* guid - unique id of the record
* uid - id of the station from API -> repetitions
* name - name of the station
* bike, spot - bool values from API ( I dont know what they meant)
* cityguid - unique id of the city
* lat,lng - cooridinates
* created, disappeared - probably the first record, when the station stopped being active ( nth like that in API)

In [11]:
places.head()

,guid,uid,name,bike,spot,cityguid,lat,lng,created,disappeared
0,2215,40,NH-Hotel / Hansa Str. 43,0,1,75,51.074133,13.738875,2019-05-23 20:08:47.487,2019-06-03 12:35:41.190
1,327398,40,NH-Hotel / Hansa Str. 43,0,1,75,51.074133,13.738875,2019-06-03 12:35:41.340,NaN
2,6997,46,4260 - Südbahnhof / Hedderichstr.,0,1,169,50.099555,8.685041,2019-05-23 20:18:33.007,2019-06-03 12:48:44.320
3,328767,46,4260 - Südbahnhof / Hedderichstr.,0,1,169,50.099555,8.685041,2019-06-03 12:48:44.443,NaN
4,2216,73,Bf. Dresden-Neustadt,0,1,75,51.065184,13.741220,2019-05-23 20:08:47.520,2019-06-03 12:35:41.350


In [7]:
places.describe()

,guid,uid,bike,spot,cityguid,lat,lng
count,5.274661e+06,5.274661e+06,5.274661e+06,5.274661e+06,5.274661e+06,5.274661e+06,5.274661e+06
mean,2.728138e+06,2.007880e+07,9.980931e-01,1.876898e-03,1.645304e+02,7.286523e+01,9.264606e+00
std,1.575025e+06,2.243333e+06,4.362587e-02,4.328251e-02,5.936417e+01,1.140517e+04,8.604564e+00
min,1.000000e+00,4.000000e+01,0.000000e+00,0.000000e+00,1.000000e+00,-5.221116e+02,-1.233831e+02
25%,1.321672e+06,1.840957e+07,1.000000e+00,0.000000e+00,1.160000e+02,5.066584e+01,7.014307e+00
50%,2.770852e+06,2.022676e+07,1.000000e+00,0.000000e+00,1.850000e+02,5.093507e+01,7.749975e+00
75%,4.089609e+06,2.195872e+07,1.000000e+00,0.000000e+00,2.060000e+02,5.126715e+01,1.106924e+01
max,5.414333e+06,2.356939e+07,1.000000e+00,1.000000e+00,2.810000e+02,9.889499e+06,6.169259e+02


Values of `lat` and `lng` are sometimes suspicious - it happens that they exceed their natural borders: [-180, 180] for longitude and [-90, 90] for latitude. Those values must be removed

In [13]:
places.loc[(places['lat'].abs()>90) | (places['lng'].abs()>180)]

,guid,uid,name,bike,spot,cityguid,lat,lng,created,disappeared
584894,572680,17112360,BIKE 66686,1,0,63,4.063939e+06,16.968674,2019-06-11 03:30:39.190,2019-06-11 05:03:51.300
1635936,1647329,18854150,BIKE 825055,1,0,175,4.851422e+06,-3.190524,2019-07-10 15:34:16.563,2019-07-10 16:35:45.380
1686126,1694020,18919704,BIKE 12552,1,0,13,9.857712e+06,8.412156,2019-07-11 22:30:09.527,2019-07-12 08:05:18.773
1730305,1741056,18972085,BIKE 38687,1,0,169,1.088466e+06,8.661751,2019-07-13 08:32:54.770,2019-07-17 18:36:36.163
1809615,1923121,19121337,BIKE 38722,1,0,169,1.236698e+06,8.651464,2019-07-16 17:50:12.913,2019-07-17 08:05:00.163
1894026,2012184,19240450,BIKE 825030,1,0,175,5.253319e+06,-3.178488,2019-07-18 22:08:29.840,2019-07-20 14:36:24.873
2063178,2180105,19479159,BIKE 38910,1,0,169,1.043745e+06,8.700949,2019-07-23 20:36:42.707,2019-07-23 21:35:29.273
2077812,2198055,19496634,BIKE 11157,1,0,12,7.915707e+06,8.762156,2019-07-24 10:00:15.670,2019-07-24 10:35:46.080
2144473,2261986,19591889,BIKE 38214,1,0,169,1.041969e+06,8.654881,2019-07-26 04:02:52.777,2019-07-26 14:07:50.990
2199069,2315431,19665274,BIKE 11416,1,0,13,9.889499e+06,8.454510,2019-07-27 15:00:18.790,2019-07-27 22:05:24.220


Since there are not many such records, it is possible to remove them from the dataset

## Bikes
- guid - unique id
- number -  a bike number
- placeguid - the station guid
- appeared - when it appears on this station
- disappeared - when the bike was taken from this station


In [15]:
bikes_data.head()

,guid,number,placeguid,appeared,disappeared
0,1495432,20,327819,2019-06-06 21:01:21.280,2019-06-10 02:03:03.823
1,6551701,20,327819,2019-07-19 12:03:22.150,2019-07-25 03:40:01.543
2,7163382,20,327819,2019-07-25 08:32:12.767,2019-07-26 01:13:25.603
3,12154095,20,4006865,2019-09-11 17:35:45.163,NaT
4,5276366,538,328956,2019-07-10 17:33:49.393,2019-07-11 15:33:53.713


# Example data

In [62]:
berlin_guid = 144
warsaw_guid = 67

## Data for Warsaw in june

In [64]:
len(warsaw_bikes)

2162410

In [65]:
warsaw_bikes = get_data_from_city(warsaw_guid)
june_index = get_time_index(datetime.strptime('01-06-2019','%d-%m-%Y'),datetime.strptime('30-06-2019','%d-%m-%Y'),60)
res = prepere_data(warsaw_bikes,june_index)
res.to_csv(os.path.join(folder, 'Warsaw_june_results.csv'))

100%|██████████| 249864/249864 [07:55<00:00, 525.76it/s]


number_of_bikes
0.0                20977
2.0                15052
1.0                14290
3.0                14043
4.0                13092
                   ...  
86.0                   1
132.0                  1
136.0                  1
99.0                   1
113.0                  1
Length: 139, dtype: int64

## Data Berlin

In [69]:
berlin_bikes = get_data_from_city(berlin_guid)
one_day_index = get_time_index(datetime.strptime('01-06-2019','%d-%m-%Y'),datetime.strptime('04-06-2019','%d-%m-%Y'),60)
res = prepere_data(berlin_bikes,one_day_index)
res.to_csv(os.path.join(folder, 'Berlin_3_days_results.csv'))

100%|██████████| 421416/421416 [06:11<00:00, 1134.91it/s]


# new data - can be ignored for now

In [128]:
with open(os.path.join(folder,'response_berlin.json'),encoding="utf8") as f:
    data = json.loads(f.read())

In [129]:
bikes_dict = {}
for station in data['countries'][0]['cities'][0]['places']:
   bikes_dict.update(dict(zip(station['bike_numbers'],[station['uid']]*station['bikes'])))


In [130]:
with open(os.path.join(folder,'response_berlin2.json'),encoding="utf8") as f:
    data2 = json.loads(f.read())

In [131]:
bikes_dict2 = {}
for station in data2['countries'][0]['cities'][0]['places']:
   bikes_dict2.update(dict(zip(station['bike_numbers'],[station['uid']]*station['bikes'])))

In [133]:
len({ k : bikes_dict2[k] for k in set(bikes_dict2) - set(bikes_dict) })

39

In [141]:
bikes_avaiable = pd.DataFrame(columns = ['number_of_bikes'])
for station in data['countries'][0]['cities'][0]['places']:
    bikes_avaiable.loc[station['uid'],'number_of_bikes'] = station['bikes']

In [150]:
max(bikes_avaiable['number_of_bikes'])

21